In [1]:
# Import libraries
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import IPython
import matplotlib.pyplot as plt
from torchaudio.utils import download_asset
import numpy as np
import pandas as pd

!pip install librosa
import librosa

!pip install opendatasets

!pip install transformers

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


cuda


In [3]:
from transformers import Wav2Vec2FeatureExtractor

class Speech_Emotion_Recognition_Model(torch.nn.Module):
    def __init__(self):
        super(Speech_Emotion_Recognition_Model, self).__init__()
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")
        self.sampling_rate = self.feature_extractor.sampling_rate
        
        self.conv1d = torch.nn.Conv1d(in_channels=60000, out_channels=70000, kernel_size=3, padding=1)
        self.relu = torch.nn.ReLU()
        
    def forward(self, input):
        features = self.feature_extractor(input).input_values
        features = torch.tensor(features)
        #features = features.squeeze(0)

        print(features.size())
        output = self.conv1d(features)
        
        output = self.relu(output)
        
        return features

model = Speech_Emotion_Recognition_Model()
expected_sampling_rate = model.sampling_rate


In [ ]:
import os
from torch.utils.data import DataLoader
from dataset import download_datasets, SpeechEmotionDataset, get_dataset_info

class AudioDataset(Dataset):
    def __init__(self, folder_path, expected_sampling_rate, audio_length):
        self.folder_path = folder_path
        self.file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.wav')]
        # todo: include type of dataset as input so that getting label works for multiple datasets
        
        self.expected_sampling_rate = expected_sampling_rate
        self.audio_length = audio_length
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        
        # todo: only do preprocessing steps once!
        
        # 03-01-01-01-01-02-01.wav -> 3rd number indicates emotion
        emotion_label = file_path.split("-")[-5]
        label = int(emotion_label)
        
        waveform, sample_rate = torchaudio.load(file_path)
        if sample_rate != self.expected_sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sample_rate, expected_sampling_rate)
        
        # Pad or truncate waveform
        waveform = waveform[0] # only use 1st channel?
        length = len(waveform)
        if length < self.audio_length: # pad
            padding = torch.zeros(self.audio_length - length)
            waveform = torch.cat([waveform, padding])

        elif length > self.audio_length: # truncate
            waveform = waveform[:self.audio_length]

        # how to deal with audio with more than 1 channel?
  
        
        return waveform, label
    
folder_path = "./datasets/ravdess-emotional-speech-audio/Actor_01/"
dataset = AudioDataset(folder_path, expected_sampling_rate, 60000)
batch_size = 32
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for data in trainloader:
    print("hi")
    
    

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import os
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

%matplotlib inline

from torchvision import datasets
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer =  optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Training loop
root_dir = './runs'
os.makedirs(root_dir, exist_ok=True)

def train_on_features(net, optimizer, device, trainloader, critrerion, epochs=1):
    if torch.cuda.is_available():
        net.cuda()
    net.train()

    loss_min = float('inf')
    for epoch in range(epochs):
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            print(labels)

            optimizer.zero_grad()

            print('forward pass')
            pred = net(inputs)
            
            print('backward pass')
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
            
            if loss < loss_min:
                torch.save(net.state_dict(), os.path.join(root_dir, "best_ser_model.pth"))


    print('Finished Training')
    return net

model = train_on_features(model, optimizer, device, trainloader, criterion)